<a href="https://colab.research.google.com/github/DiegoGarciaFrade/DC3_Group8/blob/main/Pipeline_Group_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Setup (+Installations)


In [ ]:
import logging

# define a formatter to display the messages to console (standard output)
console_formatter = logging.Formatter('%(message)s')
# console_formatter = logging.Formatter('%(levelname)s:%(module)s:%(message)s')
console_handler = logging.StreamHandler()
console_handler.setFormatter(console_formatter)
# define a logger for this notebook and attach the console handler
logger = logging.getLogger('Label-Propagation')
logger.handlers.clear()
logger.propagate = False
logger.addHandler(console_handler)
# set an appropriate level of logging for this notebook
logger.setLevel(logging.INFO)


In [ ]:
!pip install supervision
!pip install jupyter_bbox_widget
!pip install segment-anything

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.7/213.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.4/727.4 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.7 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import torch
import base64
import numpy as np
import pandas as pd
import supervision as sv
import matplotlib.pyplot as plt

from google.colab import drive
from google.colab import output
from jupyter_bbox_widget import BBoxWidget
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

In [ ]:
import torch

print(torch.__version__)

# workaround to overcome long duration needed for installing torch_scatter
#!pip install pyg_lib torch_scatter -f https://data.pyg.org/whl/torch-{torch.2.4.1+cu124}.html
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.4.0+cu124.html

2.4.1+cu121
Looking in links: https://data.pyg.org/whl/torch-2.4.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 992.4/992.4 kB 32.4 MB/s eta 0:00:00


In [ ]:
!pip install torchmetrics
!pip install timm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.5/890.5 kB 13.0 MB/s eta 0:00:00


In [ ]:
!pip install albumentations

In [ ]:
import cv2
import yaml
import pandas as pd
import albumentations
from pathlib import Path

In [ ]:
import os

In [ ]:
from google.colab import drive
from google.colab import output
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# download segment anything
!pip install 'git+https://github.com/facebookresearch/segment-anything.git'

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-sjr0geqs
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-sjr0geqs
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done


## Fileplaths setup
TO DO:
Change project root directory as needed for the DRIVE path where the project is


In [ ]:
PROJECT_ROOT_DIR = 'drive/MyDrive/Data Challenge 3/WorkingPipeline/'

In [ ]:
SAM_WEIGHTS_PATH = '/content/weights/'
SAM_WEIGHTS_FILE = 'sam_vit_h_4b8939.pth'
SAM_WEIGHTS_URL = f'https://dl.fbaipublicfiles.com/segment_anything/{SAM_WEIGHTS_FILE}'

In [ ]:
DATA_ROOT_DIR = os.path.join(PROJECT_ROOT_DIR,'jbg060_AI_for_Good_course_Shared',
                             '01_Data', 'benthic_datasets')


In [ ]:

regions_of_interest = ['PAC_AUS', 'PAC_USA', 'ATL', 'IDN_PHL']
# REGION 1 #### assigned to?
mask_folder = os.path.join(DATA_ROOT_DIR, 'mask_labels', 'reef_support', 'SEAVIEW_ATL', 'masks_stitched')
img_folder = os.path.join(DATA_ROOT_DIR, 'point_labels', 'SEAVIEW', 'ATL')
tabular_folder = os.path.join(DATA_ROOT_DIR, 'point_labels', 'SEAVIEW', 'tabular-data')

## Directory setup


In [ ]:
# Root folder in Google Drive for this project
import os
logger.info(f"Root directory for project in Google Drive: {PROJECT_ROOT_DIR}")

Root directory for project in Google Drive: drive/MyDrive/Data Challenge 3/WorkingPipeline/


In [ ]:
import sys

# add the path where point label aware superpixels and custom packags is located
sys.path.append(os.path.join(PROJECT_ROOT_DIR, 'packages'))

## Cuda and custom modules

In [ ]:
custom_modules = ['labelmate.loader', 'labelmate.visualizer',
                  'labelmate.evaluator', 'labelmate.propagator',
                  'labelmate.hypertuner', 'labelmate.patchifier',
                  ]
for module_name in custom_modules:
    logging.getLogger(module_name).setLevel(logging.INFO)
custom_modules = ['torchmetrics',
                  ]
for module_name in custom_modules:
    logging.getLogger(module_name).setLevel(logging.ERROR)
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

## Import SAM pre-trained weights

In [ ]:
# dowload the pre-trained weights mentioned in GitHub and point to that file


!wget {SAM_WEIGHTS_URL} -P {SAM_WEIGHTS_PATH}


Weights downloaded and saved to /content/weights/sam_vit_h_4b8939.pth


## DATA_ROOT_DIR Check + Results destination

In [ ]:
# Path to folder that contains images and dense masks
logger.info(f"Dataset (images and masks) will be accessed from: {DATA_ROOT_DIR}")

Dataset (images and masks) will be accessed from: drive/MyDrive/Data Challenge 3/WorkingPipeline/jbg060_AI_for_Good_course_Shared/01_Data/benthic_datasets


## Final Pipeline --- BASELINE

In [ ]:
import os
import cv2
import albumentations
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import jaccard_score, accuracy_score, precision_score, recall_score, f1_score
from segment_anything import SamPredictor, sam_model_registry
from pathlib import Path

class DataLoader:
    """
    DataLoader class for loading, processing, and transforming image, mask, and tabular data.
    """
    def __init__(self, img_folder, mask_folder, tabular_folder, transform=None, load_fraction=0.01):
        self.img_folder = img_folder
        self.mask_folder = mask_folder
        self.tabular_folder = tabular_folder
        self.transform = transform
        self.load_fraction = load_fraction

        # Load image and mask data
        self.images = sorted([f for f in os.listdir(self.img_folder) if f.endswith(('.jpg', '.png'))])
        self.masks = sorted([f for f in os.listdir(self.mask_folder) if f.endswith(('.jpg', '.png'))])

        # Print image and mask names for debugging
        print(f"Image Names: {self.images}")
        print(f"Mask Names: {self.masks}")

        # Create sets for comparison, stripping _mask suffix from masks
        img_names_set = set([os.path.splitext(f)[0] for f in self.images])
        mask_names_set = set([os.path.splitext(f)[0].replace('_mask', '') for f in self.masks])  # Ensure we handle '_mask'

        # Find corresponding pairs
        valid_img_names = img_names_set & mask_names_set
        self.images = [f for f in self.images if os.path.splitext(f)[0] in valid_img_names]
        self.masks = [f"{os.path.splitext(f)[0]}_mask.png" for f in self.images]  # Create the corresponding mask filenames

        # Limit the number of images and masks loaded
        num_images_to_load = int(len(self.images) * self.load_fraction)
        self.images = self.images[:num_images_to_load]
        self.masks = self.masks[:num_images_to_load]

        # Print filtered names for debugging
        print(f"Filtered Image Names: {self.images}")
        print(f"Filtered Mask Names: {self.masks}")

        # Create a dataset dictionary with image paths as keys and image, mask data
        self.dataset = {}
        for img_p in tqdm(self.images, desc="Loading images and masks"):
            img_name = os.path.splitext(img_p)[0]
            img = cv2.imread(os.path.join(self.img_folder, img_p))
            #################################################################################### image preprocess function goes here


            mask_path = os.path.join(self.mask_folder, img_name + '_mask.png')  # Use .png as per your example
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

            # Check if the mask is loaded correctly
            if mask is None:
                print(f"Warning: Mask for {img_name} could not be loaded. Mask path: {mask_path}")
                continue

            # Ensure images and masks have the same dimensions
            if img is not None and img.shape[:2] != mask.shape[:2]:
                print(f"Image and mask sizes do not match for {img_p}. Resizing mask.")
                mask = cv2.resize(mask, (img.shape[1], img.shape[0]))

            # Store the data
            self.dataset[img_name] = {'image': img, 'mask': mask}

        self.img_names = list(self.dataset.keys())

        # Initialize the LabelEncoder
        self.label_encoder = LabelEncoder()

        # Load tabular annotation data and add to the dataset
        for annotation_df_path in tqdm(os.listdir(self.tabular_folder), desc="Loading tabular data"):
            # region = annotation_df_path.split('_', 1)[1].split('.')[0] ############################### to reduce annotation df size
            if not annotation_df_path.startswith('annotation'):
                continue

            annotation_df = pd.read_csv(os.path.join(self.tabular_folder, annotation_df_path))
            # Print column names for debugging
            print(f"Columns in {annotation_df_path}: {annotation_df.columns}")

            # Check if the expected 'quadratid' column exists, otherwise adjust
            if 'quadratid' not in annotation_df.columns:
                raise KeyError(f"'quadratid' column not found in {annotation_df_path}. Available columns: {annotation_df.columns}")

            annotation_df['quadratid'] = annotation_df['quadratid'].astype(str)

            # Fit the LabelEncoder on the func_group to convert them into integers
            self.label_encoder.fit(annotation_df['func_group'].astype(str))  # Make sure to convert to string

            # Match tabular data with image data in the dataset
            for quadratid in annotation_df['quadratid'].unique():
                if quadratid not in self.dataset:
                    continue
                # Ensure no duplicate quadratid with different data exists
                if 'points' in self.dataset[quadratid]:
                    raise Exception("Same quadrat, different datasets!")

                # Add points and point labels to the dataset
                self.dataset[quadratid]['points'] = annotation_df[annotation_df['quadratid'] == quadratid][['x', 'y']].values
                self.dataset[quadratid]['point_labels'] = annotation_df[annotation_df['quadratid'] == quadratid]['func_group'].map({'Hard Coral': 1, 'Soft Coral': 2}).fillna(0).astype(int)
                point_labels = self.dataset[quadratid]['point_labels'].to_numpy()
                # Transform labels to integers
                self.dataset[quadratid]['point_labels'] =  point_labels

          ##  added to return quadratid's for later analysis ###########################################################################################

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):

        img_dict = self.dataset[self.img_names[idx]]
        img, mask, keypoints, point_labels = img_dict['image'], img_dict['mask'], img_dict.get('points', None), img_dict.get('point_labels', None)

        if self.transform:
            transform = self.get_resize_transform(image_size=1024)
            transformed = transform(image=img, mask=mask)
            img, mask = transformed['image'], transformed['mask']

        return img, mask, keypoints, point_labels

    @staticmethod
    def get_resize_transform(image_size=1024):
        return albumentations.Compose([
            albumentations.Resize(
                height=image_size,
                width=image_size,
                interpolation=cv2.INTER_AREA,
                p=1,
            ),
        ])


# Helper functions for SAM
def load_sam_model(sam_weights_path):
    sam = sam_model_registry['vit_h'](checkpoint=sam_weights_path)
    predictor = SamPredictor(sam)
    return predictor


# Metric calculation functions
def calculate_iou(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return jaccard_score(y_true, y_pred, average='binary')

def calculate_pixel_accuracy(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return accuracy_score(y_true, y_pred)

def calculate_dice_coefficient(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    intersection = np.sum(y_true * y_pred)
    dice_score = (2 * intersection) / (np.sum(y_true) + np.sum(y_pred))
    return dice_score

def calculate_precision(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return precision_score(y_true, y_pred, average='binary')

def calculate_recall(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return recall_score(y_true, y_pred, average='binary')

def calculate_f1_score(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return f1_score(y_true, y_pred, average='binary')


from sklearn.metrics import jaccard_score, f1_score

def calculate_iou(true_mask, pred_mask, num_classes):
    """
    Calculate Intersection over Union (IoU) for multi-class segmentation.
    """
    iou_scores = []
    for class_id in range(num_classes):
        # Flatten the masks
        true_flat = (true_mask.flatten() == class_id).astype(int)
        pred_flat = (pred_mask.flatten() == class_id).astype(int)

        # Calculate IoU for each class
        iou = jaccard_score(true_flat, pred_flat, zero_division=0)
        iou_scores.append(iou)

    return np.mean(iou_scores)  # Return average IoU

def calculate_dice(true_mask, pred_mask, num_classes):
    """
    Calculate Dice Coefficient for multi-class segmentation.
    """
    dice_scores = []
    for class_id in range(num_classes):
        # Flatten the masks
        true_flat = (true_mask.flatten() == class_id).astype(int)
        pred_flat = (pred_mask.flatten() == class_id).astype(int)

        # Calculate Dice for each class
        dice = f1_score(true_flat, pred_flat, zero_division=0)
        dice_scores.append(dice)

    return np.mean(dice_scores)  # Return average Dice

def main_pipeline(img_folder, mask_folder, tabular_folder, sam_weights_path):
    # Initialize the DataLoader with your dataset
    data_loader = DataLoader(
        img_folder=img_folder,
        mask_folder=mask_folder,
        tabular_folder=tabular_folder,
        transform=DataLoader.get_resize_transform(image_size=1024)
    )

    # Load the SAM model
    sam_predictor = load_sam_model(sam_weights_path)

    # Progress bar for the main pipeline
    progress_bar = tqdm(range(len(data_loader)), desc="Processing images")

    # Lists to store metrics
    iou_scores = []
    dice_scores = []
    pixel_scores = []
    precision_scores = []

    num_classes = 3  # Set this to the number of classes in your dataset
    mask_classes = [1, 2]

    for idx in progress_bar:
        img, mask, keypoints, point_labels = data_loader[idx]

        # Check types of img and mask
        print(f"Processing image {idx}: type(img)={type(img)}, shape(img)={img.shape}, type(mask)={type(mask)}, shape(mask)={mask.shape}")

        # Check if keypoints and point_labels are not None
        if keypoints is None or point_labels is None:
            print(f"Warning: Keypoints or point_labels are None for image {data_loader.img_names[idx]}. Skipping.")
            continue



        pred_mask = np.zeros(img.shape[:2])
        for m_class in mask_classes:
        # Use SAM to predict masks
          sam_predictor.set_image(img)
          input_points = np.array(keypoints)
          input_labels = np.where(point_labels == m_class, 1, 0)
          masks, _, _ = sam_predictor.predict(point_coords=input_points, point_labels=input_labels, multimask_output=False)
          pred_mask[masks[0] == 1] = m_class



          # Check if predicted masks are empty or None
          if pred_mask is None or pred_mask.size == 0:
              print(f"Warning: No masks predicted for image {data_loader.img_names[idx]}. Skipping.")
              continue  # Skip this iteration if there's no prediction

          # Convert predicted masks to uint8 if they are boolean
          if pred_mask.dtype == np.bool_:
              pred_mask = pred_mask.astype(np.uint8) * 255
          cv2.imwrite('/content/drive/MyDrive/please_god.jpg', pred_mask)
          print(pred_mask)


      # Update progress description
    progress_bar.set_description(f"Processing image {idx + 1}/{len(data_loader)}")

    # Print average IoU ########################################################### Added other accuracy metrics
    avg_iou = np.mean(iou_scores) if iou_scores else 0
    avg_dice = np.mean(dice_scores) if dice_scores else 0
    avg_pixel = np.mean(pixel_scores) if pixel_scores else 0
    avg_precision = np.mean(precision_scores) if precision_scores else 0

    # Print average metrics
    print(f"Average IoU: {avg_iou:.4f}")
    print(f"Average Dice: {avg_dice:.4f}")
    print(f"Average Pixel Accuracy: {avg_pixel:.4f}")
    print(f"Average Precision: {avg_precision:.4f}")

    print("Pipeline completed!")

# Example usage
img_folder = img_folder
mask_folder = mask_folder
tabular_folder = tabular_folder
sam_weights_path = SAM_WEIGHTS_PATH+SAM_WEIGHTS_FILE

# Run the pipeline
main_pipeline(img_folder, mask_folder, tabular_folder, sam_weights_path)

# Offload GPU memory after processing
torch.cuda.empty_cache()
# for TPU memory


Image Names: ['17001652802.jpg', '17001669902.jpg', '17001737702.jpg', '17001738002.jpg', '17001738102.jpg', '17001738202.jpg', '17001738302.jpg', '17001738402.jpg', '17001738502.jpg', '17001738602.jpg', '17001738702.jpg', '17001738802.jpg', '17001738902.jpg', '17001739002.jpg', '17001739102.jpg', '17001739202.jpg', '17001739302.jpg', '17001739402.jpg', '17001739502.jpg', '17001739602.jpg', '17001739702.jpg', '17001739802.jpg', '17001739902.jpg', '17001740002.jpg', '17001740102.jpg', '17001740202.jpg', '17001740302.jpg', '17001740402.jpg', '17001740502.jpg', '17001740602.jpg', '17001740702.jpg', '17001740802.jpg', '17001740902.jpg', '17001745802.jpg', '17002336001.jpg', '17002369001.jpg', '17002409801.jpg', '17004348102.jpg', '17004348202.jpg', '17004348302.jpg', '17004348402.jpg', '17004348502.jpg', '17004348602.jpg', '17004348702.jpg', '17004348802.jpg', '17004348902.jpg', '17004349002.jpg', '17004349102.jpg', '17004349202.jpg', '17004349302.jpg', '17004349402.jpg', '17004349502.jpg'

Loading images and masks:   0%|          | 0/6 [00:00<?, ?it/s]

Loading tabular data:   0%|          | 0/18 [00:00<?, ?it/s]

Columns in annotations_ATL.csv: Index(['quadratid', 'y', 'x', 'label_name', 'label', 'func_group', 'method',
       'data_set'],
      dtype='object')
Columns in annotations_IND_CHA.csv: Index(['quadratid', 'y', 'x', 'label_name', 'label', 'func_group', 'method',
       'data_set'],
      dtype='object')
Columns in annotations_PAC_SLB.csv: Index(['quadratid', 'y', 'x', 'label_name', 'label', 'func_group', 'method',
       'data_set'],
      dtype='object')
Columns in annotations_PAC_TWN.csv: Index(['quadratid', 'y', 'x', 'label_name', 'label', 'func_group', 'method',
       'data_set'],
      dtype='object')
Columns in annotations_PAC_USA.csv: Index(['quadratid', 'y', 'x', 'label_name', 'label', 'func_group', 'method',
       'data_set'],
      dtype='object')
Columns in annotations_PAC_TLS.csv: Index(['quadratid', 'y', 'x', 'label_name', 'label', 'func_group', 'method',
       'data_set'],
      dtype='object')
Columns in annotations_PAC_IDN_PHL.csv: Index(['quadratid', 'y', 'x', 'lab

Processing images:   0%|          | 0/6 [00:00<?, ?it/s]

Processing image 0: type(img)=<class 'numpy.ndarray'>, shape(img)=(1024, 1024, 3), type(mask)=<class 'numpy.ndarray'>, shape(mask)=(1024, 1024)
Predicted masks shape: (1024, 1024)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]]


KeyboardInterrupt: 

In [ ]:
help(SamPredictor)

Help on class SamPredictor in module segment_anything.predictor:

class SamPredictor(builtins.object)
 |  SamPredictor(sam_model: segment_anything.modeling.sam.Sam) -> None
 |  
 |  Methods defined here:
 |  
 |  __init__(self, sam_model: segment_anything.modeling.sam.Sam) -> None
 |      Uses SAM to calculate the image embedding for an image, and then
 |      allow repeated, efficient mask prediction given prompts.
 |      
 |      Arguments:
 |        sam_model (Sam): The model to use for mask prediction.
 |  
 |  get_image_embedding(self) -> torch.Tensor
 |      Returns the image embeddings for the currently set image, with
 |      shape 1xCxHxW, where C is the embedding dimension and (H,W) are
 |      the embedding spatial dimension of SAM (typically C=256, H=W=64).
 |  
 |  predict(self, point_coords: Optional[numpy.ndarray] = None, point_labels: Optional[numpy.ndarray] = None, box: Optional[numpy.ndarray] = None, mask_input: Optional[numpy.ndarray] = None, multimask_output: bool 

# Added CLAHE preprocessing statement

In [ ]:
################ CLAHE PREPROCESSING STATEMENT
            lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
            lab_planes = list(cv2.split(lab))
            clahe = cv2.createCLAHE(clipLimit=20.0,tileGridSize=(124,124))
            lab_planes[0] = clahe.apply(lab_planes[0])
            lab = cv2.merge(lab_planes)
            img = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)

            mask_path = os.path.join(self.mask_folder, img_name + '_mask.png')  # Use .png as per your example
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

IndentationError: unexpected indent (<ipython-input-25-2805b31ac4f9>, line 2)

# Preprocessing two . . . .

In [ ]:
def image_normalization (image):
    """
    This definition will first grayscale images to keep the contrast in the image, whenever it is normalized.
    It will normalize the image to a [0,1] scale.
    :param image: the image that will be normalized
    :return: a normalized image
    """

    #greyscale the image
    #first greyscale so that the contrast will be enhanced
    greyscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    #normalize the image
    normalized_image = cv2.normalize(greyscale_image, None , alpha=0, beta=255, norm_type= cv2.NORM_MINMAX)

    #convert back to color to be used in the SAM models
    normalized_image_color = cv2.cvtColor(normalized_image, cv2.COLOR_GRAY2RGB)

    #return the final output
    return normalized_image_color



def background_noise_removal (image):
    """
    removes the background noise of the images. Returns the image without the background noise.
    :param image: image, the path to the image
    :return: the image without background noise.
    """
    #denoising a coloured image
    cleaned_image = cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)
    return cleaned_image



In [ ]:
import os
import cv2
import albumentations
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import jaccard_score, accuracy_score, precision_score, recall_score, f1_score
from segment_anything import SamPredictor, sam_model_registry
from pathlib import Path

class DataLoader:
    """
    DataLoader class for loading, processing, and transforming image, mask, and tabular data.
    """
    def __init__(self, img_folder, mask_folder, tabular_folder, transform=None, load_fraction=0.1):
        self.img_folder = img_folder
        self.mask_folder = mask_folder
        self.tabular_folder = tabular_folder
        self.transform = transform
        self.load_fraction = load_fraction

        # Load image and mask data
        self.images = sorted([f for f in os.listdir(self.img_folder) if f.endswith(('.jpg', '.png'))])
        self.masks = sorted([f for f in os.listdir(self.mask_folder) if f.endswith(('.jpg', '.png'))])

        # Print image and mask names for debugging
        print(f"Image Names: {self.images}")
        print(f"Mask Names: {self.masks}")

        # Create sets for comparison, stripping _mask suffix from masks
        img_names_set = set([os.path.splitext(f)[0] for f in self.images])
        mask_names_set = set([os.path.splitext(f)[0].replace('_mask', '') for f in self.masks])  # Ensure we handle '_mask'

        # Find corresponding pairs
        valid_img_names = img_names_set & mask_names_set
        self.images = [f for f in self.images if os.path.splitext(f)[0] in valid_img_names]
        self.masks = [f"{os.path.splitext(f)[0]}_mask.png" for f in self.images]  # Create the corresponding mask filenames

        # Limit the number of images and masks loaded
        num_images_to_load = int(len(self.images) * self.load_fraction)
        self.images = self.images[:num_images_to_load]
        self.masks = self.masks[:num_images_to_load]

        # Print filtered names for debugging
        print(f"Filtered Image Names: {self.images}")
        print(f"Filtered Mask Names: {self.masks}")

        # Create a dataset dictionary with image paths as keys and image, mask data
        self.dataset = {}
        for img_p in tqdm(self.images, desc="Loading images and masks"):
            img_name = os.path.splitext(img_p)[0]
            img = cv2.imread(os.path.join(self.img_folder, img_p))
            #################################################################################### image preprocess function goes here
            img =   image_normalization(background_noise_removal(img))
            mask_path = os.path.join(self.mask_folder, img_name + '_mask.png')  # Use .png as per your example
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

            # Check if the mask is loaded correctly
            if mask is None:
                print(f"Warning: Mask for {img_name} could not be loaded. Mask path: {mask_path}")
                continue

            # Ensure images and masks have the same dimensions
            if img is not None and img.shape[:2] != mask.shape[:2]:
                print(f"Image and mask sizes do not match for {img_p}. Resizing mask.")
                mask = cv2.resize(mask, (img.shape[1], img.shape[0]))

            # Store the data
            self.dataset[img_name] = {'image': img, 'mask': mask}

        self.img_names = list(self.dataset.keys())

        # Initialize the LabelEncoder
        self.label_encoder = LabelEncoder()

        # Load tabular annotation data and add to the dataset
        for annotation_df_path in tqdm(os.listdir(self.tabular_folder), desc="Loading tabular data"):
            # region = annotation_df_path.split('_', 1)[1].split('.')[0] ############################### to reduce annotation df size
            if not annotation_df_path.startswith('annotation'):
                continue

            annotation_df = pd.read_csv(os.path.join(self.tabular_folder, annotation_df_path))
            # Print column names for debugging
            print(f"Columns in {annotation_df_path}: {annotation_df.columns}")

            # Check if the expected 'quadratid' column exists, otherwise adjust
            if 'quadratid' not in annotation_df.columns:
                raise KeyError(f"'quadratid' column not found in {annotation_df_path}. Available columns: {annotation_df.columns}")

            annotation_df['quadratid'] = annotation_df['quadratid'].astype(str)

            # Fit the LabelEncoder on the func_group to convert them into integers
            self.label_encoder.fit(annotation_df['func_group'].astype(str))  # Make sure to convert to string

            # Match tabular data with image data in the dataset
            for quadratid in annotation_df['quadratid'].unique():
                if quadratid not in self.dataset:
                    continue

                # Ensure no duplicate quadratid with different data exists
                if 'points' in self.dataset[quadratid]:
                    raise Exception("Same quadrat, different datasets!")

                # Add points and point labels to the dataset
                self.dataset[quadratid]['points'] = annotation_df[annotation_df['quadratid'] == quadratid][['x', 'y']].values
                point_labels = annotation_df[annotation_df['quadratid'] == quadratid]['func_group'].values

                # Transform labels to integers
                self.dataset[quadratid]['point_labels'] = self.label_encoder.transform(point_labels)

          ##  added to return quadratid's for later analysis ###########################################################################################

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):

        img_dict = self.dataset[self.img_names[idx]]
        img, mask, keypoints, point_labels = img_dict['image'], img_dict['mask'], img_dict.get('points', None), img_dict.get('point_labels', None)

        if self.transform:
            transform = self.get_resize_transform(image_size=1024)
            transformed = transform(image=img, mask=mask)
            img, mask = transformed['image'], transformed['mask']

        return img, mask, keypoints, point_labels

    @staticmethod
    def get_resize_transform(image_size=1024):
        return albumentations.Compose([
            albumentations.Resize(
                height=image_size,
                width=image_size,
                interpolation=cv2.INTER_AREA,
                p=1,
            ),
        ])


# Helper functions for SAM
def load_sam_model(sam_weights_path):
    sam = sam_model_registry['vit_h'](checkpoint=sam_weights_path)
    predictor = SamPredictor(sam)
    return predictor


# Metric calculation functions
def calculate_iou(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return jaccard_score(y_true, y_pred, average='binary')

def calculate_pixel_accuracy(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return accuracy_score(y_true, y_pred)

def calculate_dice_coefficient(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    intersection = np.sum(y_true * y_pred)
    dice_score = (2 * intersection) / (np.sum(y_true) + np.sum(y_pred))
    return dice_score

def calculate_precision(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return precision_score(y_true, y_pred, average='binary')

def calculate_recall(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return recall_score(y_true, y_pred, average='binary')

def calculate_f1_score(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return f1_score(y_true, y_pred, average='binary')


from sklearn.metrics import jaccard_score, f1_score

def calculate_iou(true_mask, pred_mask, num_classes):
    """
    Calculate Intersection over Union (IoU) for multi-class segmentation.
    """
    iou_scores = []
    for class_id in range(num_classes):
        # Flatten the masks
        true_flat = (true_mask.flatten() == class_id).astype(int)
        pred_flat = (pred_mask.flatten() == class_id).astype(int)

        # Calculate IoU for each class
        iou = jaccard_score(true_flat, pred_flat, zero_division=0)
        iou_scores.append(iou)

    return np.mean(iou_scores)  # Return average IoU

def calculate_dice(true_mask, pred_mask, num_classes):
    """
    Calculate Dice Coefficient for multi-class segmentation.
    """
    dice_scores = []
    for class_id in range(num_classes):
        # Flatten the masks
        true_flat = (true_mask.flatten() == class_id).astype(int)
        pred_flat = (pred_mask.flatten() == class_id).astype(int)

        # Calculate Dice for each class
        dice = f1_score(true_flat, pred_flat, zero_division=0)
        dice_scores.append(dice)

    return np.mean(dice_scores)  # Return average Dice

def main_pipeline(img_folder, mask_folder, tabular_folder, sam_weights_path):
    # Initialize the DataLoader with your dataset
    data_loader = DataLoader(
        img_folder=img_folder,
        mask_folder=mask_folder,
        tabular_folder=tabular_folder,
        transform=DataLoader.get_resize_transform(image_size=1024)
    )

    # Load the SAM model
    sam_predictor = load_sam_model(sam_weights_path)

    # Progress bar for the main pipeline
    progress_bar = tqdm(range(len(data_loader)), desc="Processing images")

    # Lists to store metrics
    iou_scores = []
    dice_scores = []
    pixel_scores = []
    precision_scores = []

    num_classes = 3  # Set this to the number of classes in your dataset

    for idx in progress_bar:
        img, mask, keypoints, point_labels = data_loader[idx]

        # Check types of img and mask
        print(f"Processing image {idx}: type(img)={type(img)}, shape(img)={img.shape}, type(mask)={type(mask)}, shape(mask)={mask.shape}")

        # Check if keypoints and point_labels are not None
        if keypoints is None or point_labels is None:
            print(f"Warning: Keypoints or point_labels are None for image {data_loader.img_names[idx]}. Skipping.")
            continue

        # Use SAM to predict masks
        sam_predictor.set_image(img)

        try:



            # Unpack predicted masks and any additional output (if any)
            predicted_masks, *additional_outputs = sam_predictor.predict(point_coords=keypoints, point_labels=point_labels)

            # Print shape of predicted masks for debugging
            if isinstance(predicted_masks, tuple):
                predicted_masks = predicted_masks[0]  # Extract the first element if it's a tuple

            print(f"Predicted masks shape: {predicted_masks.shape if predicted_masks is not None else 'None'}")

            # Check if the predicted masks are empty or None
            if predicted_masks is None or predicted_masks.size == 0:
                print(f"Warning: No masks predicted for image {data_loader.img_names[idx]}. Skipping.")
                continue  # Skip this iteration if there's no prediction

            # Convert predicted masks to uint8 if they are boolean
            if predicted_masks.dtype == np.bool_:
                predicted_masks = predicted_masks.astype(np.uint8) * 255  # Convert to uint8 format

            # Reduce to a single channel if necessary (e.g., take the maximum across channels)
            if predicted_masks.ndim == 3:  # If there are multiple channels
                predicted_masks = np.max(predicted_masks, axis=0)  # Or apply any other reduction logic (e.g., averaging)
                                                                   ######################################################## Might be aff evaluation

            # Ensure predicted_masks is uint8 after max operation
            predicted_masks = (predicted_masks > 0).astype(np.uint8) * 255  # Thresholding to get binary mask

            # Resize predicted mask if necessary
            if predicted_masks.shape != mask.shape:
                print(f"Resizing predicted masks from {predicted_masks.shape} to {mask.shape}")
                predicted_masks = cv2.resize(predicted_masks, (mask.shape[1], mask.shape[0]))

            # Calculate metrics
            iou = calculate_iou(mask, predicted_masks, num_classes)
            # dice = calculate_dice(mask, predicted_masks, num_classes)
            # pixel = calculate_pixel_accuracy(mask, predicted_masks)
            # precision = calculate_precision(mask, predicted_masks)
            # Store metrics
            iou_scores.append(iou)
            # dice_scores.append(dice)
            # pixel_scores.append(pixel)
            # precision_scores.append(precision)





        except Exception as e:
            print(f"Error processing image {idx}: {str(e)}")
            continue  # Skip this iteration if there's an error

        # Update progress description
        progress_bar.set_description(f"Processing image {idx + 1}/{len(data_loader)}")


    # Print average IoU ########################################################### Added other accuracy metrics
    avg_iou = np.mean(iou_scores) if iou_scores else 0
    avg_dice = np.mean(dice_scores) if dice_scores else 0
    avg_pixel = np.mean(pixel_scores) if pixel_scores else 0
    avg_precision = np.mean(precision_scores) if precision_scores else 0

    # Print average metrics
    print(f"Average IoU: {avg_iou:.4f}")
    print(f"Average Dice: {avg_dice:.4f}")
    print(f"Average Pixel Accuracy: {avg_pixel:.4f}")
    print(f"Average Precision: {avg_precision:.4f}")

    print("Pipeline completed!")



# Example usage
img_folder = img_folder
mask_folder = mask_folder
tabular_folder = tabular_folder
sam_weights_path = SAM_WEIGHTS_PATH+SAM_WEIGHTS_FILE

# Run the pipeline
main_pipeline(img_folder, mask_folder, tabular_folder, sam_weights_path)

# Offload GPU memory after processing
torch.cuda.empty_cache()



# Best model on 500 corals from everywhere

In [ ]:
import os
import cv2
import albumentations
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import jaccard_score, accuracy_score, precision_score, recall_score, f1_score
from segment_anything import SamPredictor, sam_model_registry
from pathlib import Path


class DataLoader:
    """
    DataLoader class for loading, processing, and transforming image, mask, and tabular data.
    """
    def __init__(self, img_folder, mask_folder, tabular_folder, transform=None, load_fraction=0.25):
        self.img_folder = img_folder
        self.mask_folder = mask_folder
        self.tabular_folder = tabular_folder
        self.transform = transform
        self.load_fraction = load_fraction

        # Load image and mask data
        self.images = sorted([f for f in os.listdir(self.img_folder) if f.endswith(('.jpg', '.png'))])
        self.masks = sorted([f for f in os.listdir(self.mask_folder) if f.endswith(('.jpg', '.png'))])

        #preproces image data



        # Print image and mask names for debugging
        print(f"Image Names: {self.images}")
        print(f"Mask Names: {self.masks}")

        # Create sets for comparison, stripping _mask suffix from masks
        img_names_set = set([os.path.splitext(f)[0] for f in self.images])
        mask_names_set = set([os.path.splitext(f)[0].replace('_mask', '') for f in self.masks])  # Ensure we handle '_mask'

        # Find corresponding pairs
        valid_img_names = img_names_set & mask_names_set
        self.images = [f for f in self.images if os.path.splitext(f)[0] in valid_img_names]
        self.masks = [f"{os.path.splitext(f)[0]}_mask.png" for f in self.images]  # Create the corresponding mask filenames

        # Limit the number of images and masks loaded
        num_images_to_load = int(len(self.images) * self.load_fraction)
        self.images = self.images[:num_images_to_load]
        self.masks = self.masks[:num_images_to_load]

        # Print filtered names for debugging
        print(f"Filtered Image Names: {self.images}")
        print(f"Filtered Mask Names: {self.masks}")

        # Create a dataset dictionary with image paths as keys and image, mask data
        self.dataset = {}
        for img_p in tqdm(self.images, desc="Loading images and masks"):
            img_name = os.path.splitext(img_p)[0]
            img = cv2.imread(os.path.join(self.img_folder, img_p))
            #################################################################################### image preprocess function goes here


            mask_path = os.path.join(self.mask_folder, img_name + '_mask.png')  # Use .png as per your example
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

            # Check if the mask is loaded correctly
            if mask is None:
                print(f"Warning: Mask for {img_name} could not be loaded. Mask path: {mask_path}")
                continue

            # Ensure images and masks have the same dimensions
            if img is not None and img.shape[:2] != mask.shape[:2]:
                print(f"Image and mask sizes do not match for {img_p}. Resizing mask.")
                mask = cv2.resize(mask, (img.shape[1], img.shape[0]))

            # Store the data
            self.dataset[img_name] = {'image': img, 'mask': mask}

        self.img_names = list(self.dataset.keys())

        # Initialize the LabelEncoder
        self.label_encoder = LabelEncoder()

        # Load tabular annotation data and add to the dataset
        for annotation_df_path in tqdm(os.listdir(self.tabular_folder), desc="Loading tabular data"):
            region = annotation_df_path.split('_', 1)[1].split('.')[0] ######################## added AND to reduce datasize to the region(s) selected
            if (not annotation_df_path.startswith('annotation')) and (region not in regions_of_interest):
                continue

            annotation_df = pd.read_csv(os.path.join(self.tabular_folder, annotation_df_path))
            # Print column names for debugging
            print(f"Columns in {annotation_df_path}: {annotation_df.columns}")

            # Check if the expected 'quadratid' column exists, otherwise adjust
            if 'quadratid' not in annotation_df.columns:
                raise KeyError(f"'quadratid' column not found in {annotation_df_path}. Available columns: {annotation_df.columns}")

            annotation_df['quadratid'] = annotation_df['quadratid'].astype(str)

            # Fit the LabelEncoder on the func_group to convert them into integers
            self.label_encoder.fit(annotation_df['func_group'].astype(str))  # Make sure to convert to string

            # Match tabular data with image data in the dataset
            for quadratid in annotation_df['quadratid'].unique():
                if quadratid not in self.dataset:
                    continue

                # Ensure no duplicate quadratid with different data exists
                if 'points' in self.dataset[quadratid]:
                    raise Exception("Same quadrat, different datasets!")

                # Add points and point labels to the dataset
                self.dataset[quadratid]['points'] = annotation_df[annotation_df['quadratid'] == quadratid][['x', 'y']].values
                point_labels = annotation_df[annotation_df['quadratid'] == quadratid]['func_group'].values

                # Transform labels to integers
                self.dataset[quadratid]['point_labels'] = self.label_encoder.transform(point_labels)
          ##  added to return quadratid's for later analysis ###########################################################################################

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):

        img_dict = self.dataset[self.img_names[idx]]
        img, mask, keypoints, point_labels = img_dict['image'], img_dict['mask'], img_dict.get('points', None), img_dict.get('point_labels', None)

        if self.transform:
            transform = self.get_resize_transform(image_size=1024)
            transformed = transform(image=img, mask=mask)
            img, mask = transformed['image'], transformed['mask']

        return img, mask, keypoints, point_labels

    @staticmethod
    def get_resize_transform(image_size=1024):
        return albumentations.Compose([
            albumentations.Resize(
                height=image_size,
                width=image_size,
                interpolation=cv2.INTER_AREA,
                p=1,
            ),
        ])


# Helper functions for SAM
def load_sam_model(sam_weights_path):
    sam = sam_model_registry['vit_h'](checkpoint=sam_weights_path)
    predictor = SamPredictor(sam)
    return predictor


# Metric calculation functions
def calculate_iou(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return jaccard_score(y_true, y_pred, average='binary')

def calculate_pixel_accuracy(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return accuracy_score(y_true, y_pred)

def calculate_dice_coefficient(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    intersection = np.sum(y_true * y_pred)
    dice_score = (2 * intersection) / (np.sum(y_true) + np.sum(y_pred))
    return dice_score

def calculate_precision(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return precision_score(y_true, y_pred, average='binary')

def calculate_recall(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return recall_score(y_true, y_pred, average='binary')

def calculate_f1_score(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return f1_score(y_true, y_pred, average='binary')


from sklearn.metrics import jaccard_score, f1_score

def calculate_iou(true_mask, pred_mask, num_classes):
    """
    Calculate Intersection over Union (IoU) for multi-class segmentation.
    """
    iou_scores = []
    for class_id in range(num_classes):
        # Flatten the masks
        true_flat = (true_mask.flatten() == class_id).astype(int)
        pred_flat = (pred_mask.flatten() == class_id).astype(int)

        # Calculate IoU for each class
        iou = jaccard_score(true_flat, pred_flat, zero_division=0)
        iou_scores.append(iou)

    return np.mean(iou_scores)  # Return average IoU

def calculate_dice(true_mask, pred_mask, num_classes):
    """
    Calculate Dice Coefficient for multi-class segmentation.
    """
    dice_scores = []
    for class_id in range(num_classes):
        # Flatten the masks
        true_flat = (true_mask.flatten() == class_id).astype(int)
        pred_flat = (pred_mask.flatten() == class_id).astype(int)

        # Calculate Dice for each class
        dice = f1_score(true_flat, pred_flat, zero_division=0)
        dice_scores.append(dice)

    return np.mean(dice_scores)  # Return average Dice

def main_pipeline(img_folder, mask_folder, tabular_folder, sam_weights_path):
    # Initialize the DataLoader with your dataset
    data_loader = DataLoader(
        img_folder=img_folder,
        mask_folder=mask_folder,
        tabular_folder=tabular_folder,
        transform=DataLoader.get_resize_transform(image_size=1024)
    )

    # Load the SAM model
    sam_predictor = load_sam_model(sam_weights_path)

    # Progress bar for the main pipeline
    progress_bar = tqdm(range(len(data_loader)), desc="Processing images")

    # Lists to store metrics
    iou_scores = []
    dice_scores = []

    num_classes = 3  # Set this to the number of classes in your dataset

    for idx in progress_bar:
        img, mask, keypoints, point_labels = data_loader[idx]

        # Check types of img and mask
        print(f"Processing image {idx}: type(img)={type(img)}, shape(img)={img.shape}, type(mask)={type(mask)}, shape(mask)={mask.shape}")

        # Check if keypoints and point_labels are not None
        if keypoints is None or point_labels is None:
            print(f"Warning: Keypoints or point_labels are None for image {data_loader.img_names[idx]}. Skipping.")
            continue

        # Use SAM to predict masks
        sam_predictor.set_image(img)

        try:
            # Unpack predicted masks and any additional output (if any)
            predicted_masks, *additional_outputs = sam_predictor.predict(point_coords=keypoints, point_labels=point_labels)

            # Print shape of predicted masks for debugging
            if isinstance(predicted_masks, tuple):
                predicted_masks = predicted_masks[0]  # Extract the first element if it's a tuple

            print(f"Predicted masks shape: {predicted_masks.shape if predicted_masks is not None else 'None'}")

            # Check if the predicted masks are empty or None
            if predicted_masks is None or predicted_masks.size == 0:
                print(f"Warning: No masks predicted for image {data_loader.img_names[idx]}. Skipping.")
                continue  # Skip this iteration if there's no prediction

            # Convert predicted masks to uint8 if they are boolean
            if predicted_masks.dtype == np.bool_:
                predicted_masks = predicted_masks.astype(np.uint8) * 255  # Convert to uint8 format

            # Reduce to a single channel if necessary (e.g., take the maximum across channels)
            if predicted_masks.ndim == 3:  # If there are multiple channels
                predicted_masks = np.max(predicted_masks, axis=0)  # Or apply any other reduction logic (e.g., averaging)
                                                                   ######################################################## Might be aff evaluation

            # Ensure predicted_masks is uint8 after max operation
            predicted_masks = (predicted_masks > 0).astype(np.uint8) * 255  # Thresholding to get binary mask

            # Resize predicted mask if necessary
            if predicted_masks.shape != mask.shape:
                print(f"Resizing predicted masks from {predicted_masks.shape} to {mask.shape}")
                predicted_masks = cv2.resize(predicted_masks, (mask.shape[1], mask.shape[0]))

            # Calculate metrics
            iou = calculate_iou(mask, predicted_masks, num_classes)
            dice = calculate_dice_coefficient(mask, predicted_masks, num_classes)
            pixel = calculate_pixel_accuracy(mask, predicted_masks)
            precision = calculate_precision(mask, predicted_masks)
            # Store metrics
            iou_scores.append(iou)
            dice_scores.append(dice)
            pixel_scores.append(pixel)
            precision_scores.append(precision)



        except Exception as e:
            print(f"Error processing image {idx}: {str(e)}")
            continue  # Skip this iteration if there's an error

        # Update progress description
        progress_bar.set_description(f"Processing image {idx + 1}/{len(data_loader)}")

        ####################################################################### Save predicted segmentation mask
        output_folder = os.path.join(PROJECT_ROOT_DIR, 'mask_predictions')
        quadratid = data_loader[idx]['quadratid']
        pred_mask_name = quadratid+'_BASELINE.jpg'  # Each image gets a unique name
        masks_stitched = predicted_masks # I think they get stitched by the averaging but I am not sure >>>> change this if it does not work
        cv2.imwrite(os.path.join(output_folder, image_name), masks_stitched)
        progress_bar.set_description(f"mask saved!")

    # Print average IoU
    avg_iou = np.mean(iou_scores) if iou_scores else 0
    avg_dice = np.mean(dice_scores) if dice_scores else 0
    avg_pixel = np.mean(pixel_scores) if pixel_scores else 0
    avg_precision = np.mean(precision_scores) if precision_scores else 0

    # Print average metrics
    print(f"Average IoU: {avg_iou:.4f}")
    print(f"Average Dice: {avg_dice:.4f}")
    print(f"Average Pixel Accuracy: {avg_pixel:.4f}")
    print(f"Average Precision: {avg_precision:.4f}")

    print("Pipeline completed!")



# Example usage
img_folder = img_folder
mask_folder = mask_folder
tabular_folder = tabular_folder
sam_weights_path = SAM_WEIGHTS_PATH+SAM_WEIGHTS_FILE

# Run the pipeline
main_pipeline(img_folder, mask_folder, tabular_folder, sam_weights_path)

# Offload GPU memory after processing
torch.cuda.empty_cache()